# Lent Practical 6: Adding Biology

In this practical, you will add biological processes to the 3 box ocean you created in the last practical. This will allow you to simulate the 'Biological Pump' and the 'Carbonate Pump' that you saw in Lectures 37 and 38.

This will involve:
 - Adding one new nutrient state variable, PO4, to the model
 - Parameterising the uptake of nutrients and carbon by marine organisms in the surface ocean, and the remineralisation of organic matter in the deep ocean
 - Parameterising the formation of CaCO<sub>3</sub> by calcifying organisms in the surface ocean, and the dissolution of CaCO<sub>3</sub> in the deep ocean

In [ ]:
# This code installs some required packages for the rest of the notebook. Run this once at the start - it should take <30 seconds to run.
%%capture
import os
if not os.path.exists('./OceanTools'):
    !git clone https://github.com/Quantitative-Environmental-Science/OceanTools.git

import sys
sys.path.append('./OceanTools')

!pip install cbsyst

import numpy as np
import matplotlib.pyplot as plt
from cbsyst import Csys  # this is for carbon chemistry calculations

## A 3 Box Ocean: Adding Biology

### Biology

To include biology we must first include a nutrient that controls the patterns of productivity in the surface ocean.
We'll use phosphorous (P), which is present in the ocean as $PO_4$ and is limiting in large parts of the ocean and is straightforwardly related to carbon via the stoichiometry $106C:1P$, known as the 'Redfield Ratio'.

We will model P as a conservative tracer that is consumed in the surface boxes, and released in the deep box.
As with other processes in the model, we'll describe the consumption of P with a characteristic timescale, $\tau_P$, which describes how long P stays in the surface box before being consumed:

$$
\mathrm{\frac{dP_{bio}}{dt} = \frac{V [P]}{\tau_P}}
$$

Which gives the total moles of P consumed in the surface box ($[P]$ is concentration of phosphorous).
This consumed P is then transported by particle sinking to the underlying deep box.
We can combine this with P transport to give:

$$
\begin{align}
\frac{d[P]_L}{dt} &= \left. \left( Q (P_D - P_L) - \frac{V_L}{\tau_L} (P_L - P_D) - \frac{V_L P_L}{\tau_{PL}} \right) \middle/ V_L \right. \\
\frac{d[P]_H}{dt} &= \left. \left( Q (P_L - P_H) - \frac{V_H}{\tau_H} (P_H - P_D) - \frac{V_H P_H}{\tau_{PH}} \right) \middle/ V_H \right. \\
\frac{d[P]_D}{dt} &= \left. \left( Q (P_H - P_D) - \frac{V_H}{\tau_H} (P_D - P_H) - \frac{V_L}{\tau_L} (P_D - P_L) + \frac{V_H P_H}{\tau_{PH}} + \frac{V_L P_L}{\tau_{PL}} \right) \middle/ V_D \right. \\
\end{align}
$$

The impact of organic matter breakdown on DIC and TA are then given by the extended Redfield Ratio, which comes from the stoichiometry of the organic matter:

$$
\mathrm{\underbrace{106 CO_2}_{carbon~dioxide} + \underbrace{16 NO_3 + H_3PO_4}_{nutrients} + 78 H_2O + \underbrace{18H^+}_{alkalinity} \rightleftharpoons \underbrace{C_{106}H_{175}O_{42}N_{16}P}_{organic~matter} + \underbrace{150 O_2}_{oxygen}}
$$

Which, within the parameters of our model, can be simplified to:

$$
\mathrm{P + 106 DIC - 18 TA \rightleftharpoons [organic~matter]}
$$

The change in DIC due to biological productivity is therefore given by:
$$
\mathrm{\frac{d[DIC]_{bio}}{dt} = 106 \frac{dP_{bio}}{dt}}
$$
and the change in TA is given by:
$$
\mathrm{\frac{d[TA]_{bio}}{dt} = -18 \frac{dP_{bio}}{dt}}
$$

#### Calcification

To include calcification, we will assign a fraction of the organic matter production that also forms $CaCO_3$:

$$
\mathrm{f_{CaCO3} = \frac{DIC_{CaCO3}}{DIC_{organic}}}
$$

The addition or removal of $\mathrm{[CO_3^{2-}]}$ changes DIC by a factor of 1, and TA by a factor of 2 (because the carbonate ion has a -2 charge).
We can therefore write this in our model as:

$$
\begin{align}
\mathrm{\frac{dDIC_{calc}}{dt}} &= \mathrm{f_{CaCO3} \frac{dDIC_{bio}}{dt} }\\
\mathrm{\frac{dTA_{calc}}{dt}} &= \mathrm{2 \frac{dDIC_{calc}}{dt}}
\end{align}
$$

### Model Setup

#### Global Variables
The cell below contains a number of key variables that you will use to construct a three-box model of ocean circulation driven by changes in air temperature and salinity inputs.

In [ ]:
# global variables
V_ocean = 1.34e18  # volume of the ocean in m3
SA_ocean = 358e12  # surface area of the ocean in m2
fSA_hilat = 0.15  # fraction of ocean surface area in 'high latitude' box

# variables used to calculate Q
Q_alpha = 1e-4
Q_beta = 7e-4
Q_k = 8.3e17

# salinity balance - the total amount of salt added or removed to the surface boxes
Fw = 0.1  # low latitude evaporation - precipitation in units of m yr-1
Sref = 35  # reference salinity in units of g kg-1
E = Fw * SA_ocean * (1 - fSA_hilat) * Sref  # amount of salt removed from the low latitude box,  g kg-1 yr-1, ~ kg m-3 yr-1

#### Set Up Boxes

The dictionaries you used to describe the boxes in the last practical are below.
Add `PO4`, `tau_PO4` and `f_CaCO3` to the boxes, as appropriate, and set the initial values as described in the table below

| Variable | `lolat` | `hilat` | `deep` | units |
|---|---|---|---|---|
| `PO4` | $0.182 \times 10^{-3}$ | $1.68 \times 10^{-3}$ | $2.38 \times 10^{-3}$ | mol m-3 |
| `tau_PO4` | 2 | 3 | - | yr |
| `f_CaCO3` | 0.2 | 0.1 | - | unitless |


In [ ]:
# set up boxes

total_DIC = 38900e15 / 12  # mol C
avg_DIC = total_DIC / V_ocean

total_TA = 3.1e18  # mol TA
avg_TA = total_TA / V_ocean

init_hilat = {
    'name': 'hilat',
    'depth': 200,  # box depth, m
    'SA': SA_ocean * fSA_hilat,  # box surface area, m2 
    'T': 3.897678,  # initial water temperature, Celcius
    'S': 34.37786,  # initial salinity
    'T_atmos': 0.,  # air temperature, Celcius
    'tau_M': 100.,  # timescale of surface-deep mixing, yr
    'tau_T': 2.,  # timescale of temperature exchange with atmosphere, yr
    'E': -E,  # salt added due to evaporation - precipitation, kg m-3 yr-1
    'tau_CO2': 2.,  # timescale of CO2 exchange, yr
    'DIC': 2.32226,  # Dissolved Inorganic Carbon concentration, mol m-3
    'TA': avg_TA,  # Total Alkalinity, mol m-3
    # TODO: Add new variables here
}
init_hilat['V'] = init_hilat['SA'] *  init_hilat['depth']  # box volume, m3

init_lolat = {
    'name': 'lolat',
    'depth': 100,  # box depth, m
    'SA': SA_ocean * (1 - fSA_hilat),  # box surface area, m2 
    'T': 23.60040,  # initial water temperature, Celcius
    'S': 35.37898,  # initial salinity
    'T_atmos': 25.,  # air temperature, Celcius
    'tau_M': 250.,  # timescale of surface-deep mixing, yr
    'tau_T': 2.,  # timescale of temperature exchange with atmosphere, yr
    'E': E,  # salinity balance, PSU m3 yr-1
    'tau_CO2': 2.,  # timescale of CO2 exchange, yr
    'DIC': 2.26201,  # Dissolved Inorganic Carbon concentration, mol m-3
    'TA': avg_TA,  # Total Alkalinity, mol m-3
    # TODO: Add new variables here
}
init_lolat['V'] = init_lolat['SA'] *  init_lolat['depth']  # box volume, m3

init_deep = {
    'name': 'deep',
    'V': V_ocean - init_lolat['V'] - init_hilat['V'],  # box volume, m3
    'T': 5.483637,  # initial water temperature, Celcius
    'S': 34.47283,  # initial salinity
    'DIC': 2.32207,  # Dissolved Inorganic Carbon concentration, mol m-3
    'TA': avg_TA,  # Total Alkalinity, mol m-3
    # TODO: Add new variables here
}

init_atmos = {
    'name': 'atmos',
    'mass': 5.132e18,  # kg
    'moles_air': 1.736e20,  # moles
    'moles_CO2': 850e15 / 12,  # moles
    'GtC_emissions': 0.0  # annual emissions of CO2 into the atmosphere, GtC
}
init_atmos['pCO2'] = init_atmos['moles_CO2'] / init_atmos['moles_air'] * 1e6

### Add `PO4` and Calcification to Model Loop

The model loop you wrote in the last practical is below. Add PO4, biology and calcification to the loop in the places indicated.

In [ ]:
def ocean_model(lolat, hilat, deep, atmos, tmax, dt):
    """Run the ocean model for a given time period and return the results for each box.

    Parameters
    ----------
    lolat, hilat, deep, atmos : dict
        dictionaries containing the box properties
    tmax : int or float
        The maximum time to run the model for (yr)
    dt : float
        The time step of the model (yr)

    Returns
    -------
    tuple of (time, lolat, hilat, deep)
    """
    
    # create the time scale for the model
    time = np.arange(0, tmax + dt, dt)
    
    # identify which variables will change with time
    model_vars = ['T', 'S', 'DIC', 'TA']  # --> TODO: Add PO4 here 
    atmos_model_vars = ['moles_CO2', 'pCO2']

    # create copies of the input dictionaries so we don't modify the originals
    lolat = lolat.copy()
    hilat = hilat.copy()
    deep = deep.copy()
    atmos = atmos.copy()
    
    # turn all time-evolving variables into arrays containing the start values
    for box in [lolat, hilat, deep]:
        for k in model_vars:
            box[k] = np.full(time.shape, box[k])
    for k in atmos_model_vars:
        atmos[k] = np.full(time.shape, atmos[k])
    if isinstance(atmos['GtC_emissions'], (int, float)):
        atmos['GtC_emissions'] = np.full(time.shape, atmos['GtC_emissions'])
    
    # calculate initial surface carbon chemistry in the surface boxes using Csys, and store a few key variables - CO2, pH, pCO2 and K0
    for box in [lolat, hilat]:
        csys = Csys(
            TA=box['TA'],
            DIC=box['DIC'],
            T_in=box['T'], S_in=box['S'],
            unit='mmol' # we specify mmol here because mol m-3 is the same as mmol L-1, and Csys works in L-1
            )
        box['CO2'] = csys.CO2
        box['pH'] = csys.pHtot
        box['pCO2'] = csys.pCO2
        box['K0'] = csys.Ks.K0
    
    # Create a dictionary to keep track of the fluxes calculated at each step    
    fluxes = {}

    for i in range(1, time.size):
        last = i - 1  # index of last model step
        
        # calculate circulation flux, Q
        dT = lolat['T'][last] - hilat['T'][last]
        dS = lolat['S'][last] - hilat['S'][last]
        Q = Q_k * (Q_alpha * dT - Q_beta * dS)
        
        # calculate mixing fluxes for model variables
        for var in model_vars:
            # TODO: Nothing! If you've added PO4 to the model_vars list above, the mixing and circulation of PO4 is calculated automatically here
            fluxes[f'Q_{var}_deep'] = Q * (hilat[var][last] - deep[var][last]) * dt  # amount dt-1
            fluxes[f'Q_{var}_hilat'] = Q * (lolat[var][last] - hilat[var][last]) * dt  # amount dt-1
            fluxes[f'Q_{var}_lolat'] = Q * (deep[var][last] - lolat[var][last]) * dt  # amount dt-1
            
            fluxes[f'vmix_{var}_hilat'] = hilat['V'] / hilat['tau_M'] * (hilat[var][last] - deep[var][last]) * dt  # amount dt-1
            fluxes[f'vmix_{var}_lolat'] = lolat['V'] / lolat['tau_M'] * (lolat[var][last] - deep[var][last]) * dt  # amount dt-1
        
        # calculate surface-specific fluxes
        for box in [hilat, lolat]:
            boxname = box['name']
            # temperature exchange with atmosphere
            fluxes[f'dT_{boxname}'] = box['V'] / box['tau_T'] * (box['T_atmos'] - box['T'][last]) * dt  # V * degrees dt-1
            # CO2 exchange with atmosphere
            fluxes[f'dCO2_{boxname}'] = box['V'] / box['tau_CO2'] * (box['CO2'][last] - 1e-3 * atmos['pCO2'][last] * box['K0'][last]) * dt  # mol dt-1
            
            # TODO: Caulculate export fluxes of PO4, DIC and TA by from the surface ocean boxes by biology and calcitification.
            
            
        fluxes['dCO2_emissions'] = atmos['GtC_emissions'][last] * 1e15 / 12 * dt  # mol dt-1

        # update deep box
        for var in model_vars:
            if var in ['T', 'S']:
                # NOTE: 'if' statment is here because 'T' and 'S' are set by mixing alone, 
                # whereas 'DIC', 'TA' and 'PO4' now have export terms, so have to be treated
                # differently.
                deep[var][i] = deep[var][last] + (
                    fluxes[f'Q_{var}_deep'] + fluxes[f'vmix_{var}_hilat'] + fluxes[f'vmix_{var}_lolat']
                ) / deep['V']
            else:
                # TODO update DIC, TA and PO4 in the deep ocean here

        # update surface boxes
        for box in [hilat, lolat]:
            boxname = box['name']
            box['S'][i] = box['S'][last] + (fluxes[f'Q_S_{boxname}'] - fluxes[f'vmix_S_{boxname}'] + box['E'] * dt) / box['V']  # PSU dt-1
            box['T'][i] = box['T'][last] + (fluxes[f'Q_T_{boxname}'] - fluxes[f'vmix_T_{boxname}'] + fluxes[f'dT_{boxname}']) / box['V']  # degrees dt-1
            
            # TODO: modify the DIC and TA in the surface boxes to include the new export processes
            box['DIC'][i] = box['DIC'][last] + (fluxes[f'Q_DIC_{boxname}'] - fluxes[f'vmix_DIC_{boxname}'] - fluxes[f'dCO2_{boxname}']) / box['V']  # mol m-3 dt-1
            box['TA'][i] = box['TA'][last] + (fluxes[f'Q_TA_{boxname}'] - fluxes[f'vmix_TA_{boxname}']) / box['V']  # mol m-3 dt-1
            
            # TODO: update surface ocean PO4 here.

            # update carbon speciation
            csys = Csys(
                TA=box['TA'][i],
                DIC=box['DIC'][i],
                T_in=box['T'][i], S_in=box['S'][i],
                unit='mmol'
                )
            box['CO2'][i] = csys.CO2[0]
            box['pH'][i] = csys.pHtot[0]
            box['pCO2'][i] = csys.pCO2[0]
            box['K0'][i] = csys.Ks.K0
        
        # update atmosphere
        atmos['moles_CO2'][i] = atmos['moles_CO2'][last] + fluxes['dCO2_hilat'] + fluxes['dCO2_lolat'] + fluxes['dCO2_emissions']
        atmos['pCO2'][i] = 1e6 * atmos['moles_CO2'][i] / atmos['moles_air']

    return time, lolat, hilat, deep, atmos

## Run your model! 

Run your model for 1000 years with a time step of 1 year, and look at how the carbon system evolves in the model.

In [ ]:
time, lolat, hilat, deep, atmos = ocean_model(init_lolat, init_hilat, init_deep, init_atmos, 1000, 1)

Look at the results:

In [ ]:
from tools import plot
from tools.helpers import get_last_values

fig, axs = plot.boxes(time, ['DIC', 'TA', 'pCO2'], lolat, hilat, deep, atmos)

for k, v in get_last_values(hilat, lolat, atmos).items():
    print(k, v['pCO2'])

You can compare the output of your model to the reference model:

```python
from tools.working import ocean_model_bio
```

In [ ]:
from tools.working import ocean_model_bio

time, lolat, hilat, deep, atmos = ocean_model(init_lolat, init_hilat, init_deep, init_atmos, 1000, 0.5)

fig, axs = plot.boxes(time, ['DIC', 'TA', 'pCO2'], lolat, hilat, deep, atmos)

for k, v in get_last_values(hilat, lolat, atmos).items():
    print(k, v['pCO2'])

## Think

1. Compare the pCO2 in the atmosphere and surface ocean boxes with the output from the last practical. What has changed?
2. Compare the response of DIC and TA between this model and the last. What has changed?

## Experiment: Climate Change Interactions

There are a lot of interacting processes in this model, which can make it difficult to work out the influence each is having. To pick this apart you're going to conduct a few experiments. 

### 1. Ocean Acidification

Ocean acidification is likely to reduce the production of calcium carbonate shells in the surface ocean. Simulate this by halving the `f_CaCO3` parameter in both surface boxes. 

How does this change equilibrium pCO2? Why does it change like this? What does this mean for the impact of ocean acidification?

In [ ]:
# Do it here

### 2. Ballasting Feedback

If there is less calcification, biological matter sinks more slowly so will be exported to the deep ocean less efficiently.

Keep `f_CaCO3` at its reduced value from Experiment 1, and reduce the export efficiency of the biological pump by increasing `tau_PO5` by a factor of 2 in each box.

How does this change equilibrium pCO2?

In [ ]:
# Do it here

## Lab Report Question 5

Conduct a CO2 release experiment in the ocean using each of the model configuration you examined above (i.e. the standard model, the model with reduced calcification, and the model with reduced calcification and reduced ballasting). Release 8 GtC a year between years 200-400.

Create a plot showing the response of pCO2 in all three models to this CO2 release.

1. Compare the equilibrium pCO2 in the three models compared to their equilibrium pCO2 in the absence of the CO2 release. How does the partitioning of C between the ocean and atmosphere change in each model? 
2. What mechanisms are responsible for this?

In [ ]:
# Do it here